In [1]:
import sys
sys.path.append(r'../train')

import tensorflow as tf

from tf_memn_classify import tmv_tf_memn_classify

csv_dump = True
batch_size = 100
# epochs = 20
epochs = 1
#epochs = 50

dependent_var = r'Definition-Score'
task_word = r'Definition'
number_class = 3
key_word = r'TF_MEMN2N-Def-PRE-POST-All_Jupyter'

C:\Users\macks\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\macks\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\macks\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\macks\AppData\Roaming\Python\Python36\site-

In [2]:
memnd = tmv_tf_memn_classify(r'../data/')
memnd.load_data(r'Serialized-Def-ELVA.PILOT.PRE-TEST.csv', dependent_var, [0, 1], task_word)
# memnd.load_data(r'Serialized-Def-ELVA.PILOT.POST-TEST.csv', dependent_var, [0, 1], task_word)

In [3]:
memnd.perform_modeling(memnd.df_ac_modeling_values, key_word, csv_dump, number_class, epochs, batch_size)

Building model...


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Training model...



epoch: 0  validation loss: 0.49703518  validation accuracy: 0.86399615


https://josehoras.github.io/saving-a-model-in-tensorflow/

In [4]:
memnd.sess.close()

# saver = tf.train.Saver()
saver = tf.train.import_meta_graph('../train/test-log-MEMN2N/my_model.ckpt-19.meta')
sess = tf.Session()

A = sess.graph.get_tensor_by_name('A:0')
B = sess.graph.get_tensor_by_name('B:0')
C = sess.graph.get_tensor_by_name('C:0')
W = sess.graph.get_tensor_by_name('W:0')
b = sess.graph.get_tensor_by_name('b_1:0')

saver.restore(sess, '../train/test-log-MEMN2N/my_model.ckpt-19')
# saver.restore(sess, tf.train.latest_checkpoint('../train/log/'))

memnd.sess = sess

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ../train/test-log-MEMN2N/my_model.ckpt-19


In [5]:
tf.all_variables()

Instructions for updating:
Please use tf.global_variables instead.


[<tf.Variable 'A:0' shape=(961, 64) dtype=float32_ref>,
 <tf.Variable 'B:0' shape=(961, 64) dtype=float32_ref>,
 <tf.Variable 'C:0' shape=(961, 29) dtype=float32_ref>,
 <tf.Variable 'LSTM/basic_lstm_cell/kernel:0' shape=(124, 128) dtype=float32_ref>,
 <tf.Variable 'LSTM/basic_lstm_cell/bias:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'W:0' shape=(32, 3) dtype=float32_ref>,
 <tf.Variable 'b_1:0' shape=(3,) dtype=float32_ref>,
 <tf.Variable 'beta1_power:0' shape=() dtype=float32_ref>,
 <tf.Variable 'beta2_power:0' shape=() dtype=float32_ref>,
 <tf.Variable 'A/Adam:0' shape=(961, 64) dtype=float32_ref>,
 <tf.Variable 'A/Adam_1:0' shape=(961, 64) dtype=float32_ref>,
 <tf.Variable 'B/Adam:0' shape=(961, 64) dtype=float32_ref>,
 <tf.Variable 'B/Adam_1:0' shape=(961, 64) dtype=float32_ref>,
 <tf.Variable 'C/Adam:0' shape=(961, 29) dtype=float32_ref>,
 <tf.Variable 'C/Adam_1:0' shape=(961, 29) dtype=float32_ref>,
 <tf.Variable 'LSTM/basic_lstm_cell/kernel/Adam:0' shape=(124, 128) dtype=f

In [6]:
memnd.perform_prediction(memnd.df_ac_modeling_values, number_class)
memnd.evaluate_prediction(key_word)

Recall                      0.9957
Precision                   0.9957
F1                          0.9957
Kappa                       0.9823
Quadratic Weighted Kappa    0.9871
dtype: float64
Confusion Matrix:
      0    1    2
0  3595    5    2
1     1  354    7
2     1    2  202


In [7]:
from sklearn.preprocessing import OneHotEncoder

df_ac_predict_target = memnd.df_ac_modeling_values.loc[:,[memnd.dependent_var]]
y_train = df_ac_predict_target.transpose().values[0]
y_matrix_train = y_train.reshape(len(y_train),1)
ohe = OneHotEncoder(categorical_features=[0])
y_ohe_train = ohe.fit_transform(y_matrix_train).toarray()
ans_tokens_vector = memnd.vectorize_tokens(list(memnd.df_ac_modeling_values['Anser_Tokens']),
                                                  memnd.ans_maxlen)
ans_ex_tokens_vector = memnd.vectorize_tokens(list(memnd.df_ac_modeling_values['Anser_example_Tokens']),
                                                  memnd.ans_ex_maxlen)

In [8]:
predictions = sess.run(memnd.y, feed_dict={
            memnd.x: ans_ex_tokens_vector,
            memnd.q: ans_tokens_vector,
            memnd.a: y_ohe_train,
            memnd.n_batch: len(ans_ex_tokens_vector)
})
predictions

array([[1.15065295e-02, 9.74352479e-01, 1.41408797e-02],
       [3.93880950e-03, 9.88043308e-01, 8.01787619e-03],
       [1.97967701e-03, 1.26843244e-01, 8.71177137e-01],
       ...,
       [9.99561250e-01, 3.53961776e-04, 8.46716939e-05],
       [9.99448478e-01, 4.06263978e-04, 1.45183585e-04],
       [9.99473035e-01, 4.38389601e-04, 8.86859707e-05]], dtype=float32)

In [9]:
import nltk.data

def get_tokens(answer_str_list):
    list_cntnt = list(answer_str_list)
    sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

    tokens_all = []
    for x in list_cntnt:
        tokens = []
        sentences = sent_detector.tokenize(x.strip())
        for y in sentences:
            tokens += nltk.word_tokenize(y)
        tokens_all = tokens_all + [tokens]
        
    return tokens_all

In [10]:
import pandas as pd
import numpy as np
answer_clm = 'Definition-Answer'
answer_ex_clm = task_word

df_response_answer_ex = pd.read_csv('../data/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv', encoding= 'latin1')
df_response_answer_ex = df_response_answer_ex.set_index(r'Student_Question_Index')

ans_tokens = get_tokens(df_response_answer_ex[answer_clm].values)
ans_ex_tokens = get_tokens(df_response_answer_ex[answer_ex_clm].values)

df_ac_modeling_values = pd.DataFrame({'Anser_Tokens': ans_tokens,
                                        'Anser_example_Tokens': ans_ex_tokens})

ans_tokens_vector = memnd.vectorize_tokens(list(df_ac_modeling_values['Anser_Tokens']),
                                          memnd.ans_maxlen)
ans_ex_tokens_vector = memnd.vectorize_tokens(list(df_ac_modeling_values['Anser_example_Tokens']),
                                          memnd.ans_ex_maxlen)

print(ans_tokens)
print(ans_tokens_vector)
print(ans_ex_tokens)
print(ans_ex_tokens_vector)

df_ac_predict_target = df_response_answer_ex.loc[:,[dependent_var]]
y_test = df_ac_predict_target.transpose().values[0]
y_matrix_test = y_test.reshape(len(y_test),1)
y_ohe_test = ohe.fit_transform(y_matrix_test).toarray()

answer_len = len(df_response_answer_ex)
prediction = sess.run(memnd.y, feed_dict={
    memnd.x: ans_ex_tokens_vector,
    memnd.q: ans_tokens_vector,
    memnd.a: y_ohe_test,
    memnd.n_batch: answer_len
})

predict_res = np.zeros(answer_len, dtype=np.int)
for i in range(len(prediction)):
    predict_res[i] =  np.argmax(prediction[i])

df_ac_classified = pd.DataFrame(np.array(predict_res, dtype=np.int64), None,[r'Score_Class'])
df_ac_classified.index.name = r'AC_Doc_ID'
df_ac_classified

[['To', 'invent', 'stuff'], ['Someone', 'that', 'saves', 'a', 'person'], ['Something', 'that', 'is', 'not', 'possible'], ['When', 'something', 'explodes']]
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0 140 504 832]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0 116 861 749 160 665]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0 118 861 510 626 678]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0 149 805 380]]
[['A', 'person', 'who', 'creates', 'something', 'new', 'that', 'has', 'never', 'been', 'made', 'before'], ['Someone', 'you', 'admire', 'because', 'they', 'have', 'done', 'an', 'action', 'that', 'is', 'brave', 'or', 'new', 'or', 'good'], ['Describes', 'something', 'that', 'is', 'very', 'difficult', 'or', 'can', 'not', 'happen'], ['It', 'means', 'to', 'explod

,Score_Class
AC_Doc_ID,
0,1
1,1
2,1
3,2
